<a href="https://colab.research.google.com/github/pela-andrea/people-analytics-case/blob/fix%2Fajuste-script-glossario/scripts/01_exploracao_glossario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Estudo de Base de Dados RH – Case Técnico


Este notebook tem como objetivo documentar o processo de **exploração, limpeza e transformação de dados** a partir de uma base fictícia de Recursos Humanos. O material faz parte da entrega de um case técnico, solicitado com foco nas seguintes competências:

-  **Documentação clara e detalhada**
-  **Versionamento de código**
- **Análise de dados e extração de insights**
-  **Storytelling e visualização com Power BI**

---

##Estrutura do projeto

Este notebook é o primeiro de uma sequência que compõe o repositório **rh-data-case-andrea-pela**, disponível no [GitHub](https://github.com/pela-andrea/people-analytics-case). A estrutura geral do projeto está organizada em pastas com arquivos de dados, scripts de tratamento, dashboard e documentação técnica.

---

##Objetivo deste notebook

Neste notebook, você encontrará:

- Leitura da base original;
- Entendimento e descrição das colunas;
- Identificação e tratamento de dados inconsistentes ou faltantes;
- Inferência e conversão de tipos de dados;
- Criação do glossário com:
    - Nome da coluna original;
    - Nome amigável;
    - Tipo de dado;
    - Observações relevantes;
- Salvamento do glossário como `.md` ou `.csv`.

---

##Observações

- A base utilizada **não contém dados reais**.
- Todos os arquivos utilizados e gerados estão disponíveis no repositório.
- O projeto será documentado e versionado via GitHub para avaliação.

---

*Autora: [Andrea Pela](https://www.linkedin.com/in/pela-andrea/)   
*GitHub: [@pela-andrea](https://github.com/pela-andrea)*  
*Data: Julho/2025*


#**Etapa 01 - Exploração dos dados**

##Declaração das LIBS

In [93]:
# Principais
import pandas as pd
import numpy as np

#Para visualização
import matplotlib.pyplot as plt
import seaborn as sns

#Para tratamento de datas e warnings
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

#Para salvar os arquivos gerados, localmente
from google.colab import files

##Configuração do pandas

In [94]:
#Exibir todas as colunas do dataframe
pd.set_option('display.max_columns', None)

##Importação e Extração dos dados

In [95]:
#Importando a base original diretamente do repositório GitHub
url = 'https://raw.githubusercontent.com/pela-andrea/people-analytics-case/main/data/base_headcount.csv'


df = pd.read_csv(url)

##Identificação dos dados

In [103]:
#Análise descritiva dos dados
df.describe()

,salary,absence_days,sick_days,vacation_days_taken,bank_hours,overtime_hours,tardiness_count,number_of_dependents,performance_rating,bonus_percentage,tenure_years
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,6008.196118,2.004200,3.005700,14.997260,50.315640,49.940300,5.009000,1.001760,3.006220,10.031756,7.966760
std,1976.890182,1.416539,1.729152,3.861434,86.698049,29.195358,2.242744,1.002026,1.412325,4.884612,4.334095
min,1500.000000,0.000000,0.000000,3.000000,-100.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.500000
25%,4642.750000,1.000000,2.000000,12.000000,-24.000000,24.000000,3.000000,0.000000,2.000000,6.630000,4.220000
50%,6003.480000,2.000000,3.000000,15.000000,50.000000,50.000000,5.000000,1.000000,3.000000,9.970000,7.930000
75%,7354.120000,3.000000,4.000000,18.000000,125.000000,75.000000,6.000000,2.000000,4.000000,13.310000,11.720000
max,14958.170000,11.000000,13.000000,32.000000,200.000000,100.000000,15.000000,8.000000,5.000000,32.440000,15.500000


In [104]:
#Volume do DataSet
df.shape

(50000, 31)

In [105]:
#Identificação das primeiras linhas
df.head()

,employee_id,job_title,department,location,salary,hire_date,termination_date,is_active,absence_days,sick_days,vacation_days_taken,bank_hours,overtime_hours,tardiness_count,gender,marital_status,number_of_dependents,education_level,performance_rating,bonus_percentage,shift,contract_type,cost_center,compliance_status,health_plan,email,tenure_years,probation_completed,manager_id,last_promotion_date,last_training_date
0,EMP_00001,Gerente de RH,Financeiro,Salvador,6993.43,2022-02-26,NaN,True,3,2,13,52,20,6,Other,Widowed,4,Bachelors,5,7.58,Morning,Permanent,CC013,Compliant,Standard,emp_00001@company.com,3.34,True,EMP_00495,NaN,2024-07-04
1,EMP_00002,Engenheiro de Dados,Marketing,Belo Horizonte,5723.47,2023-03-04,NaN,True,1,1,14,-80,86,6,Male,Married,1,Bachelors,5,8.72,Morning,Temporary,CC026,Compliant,Basic,emp_00002@company.com,2.33,True,EMP_00345,2024-07-21,2024-02-01
2,EMP_00003,Desenvolvedor Python,Operações,Salvador,7295.38,2010-09-01,NaN,True,3,5,25,149,23,1,Female,Divorced,1,Masters,4,7.93,Morning,Permanent,CC010,Compliant,Basic,emp_00003@company.com,14.84,True,EMP_00306,2014-10-18,2024-01-11
3,EMP_00004,Analista Financeiro,Operações,Porto Alegre,9046.06,2020-02-26,NaN,True,2,1,21,-92,22,5,Male,Married,0,Bachelors,3,5.27,Morning,Permanent,CC008,Compliant,Standard,emp_00004@company.com,5.35,True,EMP_00971,NaN,2025-01-05
4,EMP_00005,Engenheiro de Dados,Recursos Humanos,Curitiba,5531.69,2011-06-22,NaN,True,1,2,15,93,72,3,Male,Single,0,Bachelors,2,10.95,Afternoon,Permanent,CC013,Compliant,Premium,emp_00005@company.com,14.03,True,EMP_00024,NaN,2025-01-19


In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   employee_id           50000 non-null  object 
 1   job_title             50000 non-null  object 
 2   department            50000 non-null  object 
 3   location              50000 non-null  object 
 4   salary                50000 non-null  float64
 5   hire_date             50000 non-null  object 
 6   termination_date      10026 non-null  object 
 7   is_active             50000 non-null  bool   
 8   absence_days          50000 non-null  int64  
 9   sick_days             50000 non-null  int64  
 10  vacation_days_taken   50000 non-null  int64  
 11  bank_hours            50000 non-null  int64  
 12  overtime_hours        50000 non-null  int64  
 13  tardiness_count       50000 non-null  int64  
 14  gender                50000 non-null  object 
 15  marital_status     

In [107]:
#Verificando a quantidade de linhas duplicadas
duplicatas = df[df.duplicated()]

soma_duplicatas = duplicatas.groupby(duplicatas.columns.tolist()).sum()

print(soma_duplicatas)

Empty DataFrame
Columns: []
Index: []


In [108]:
#Porcentagem de valores nulos em todas as variáveis
porcentagem_nulos = (df.isnull().sum() / len(df)) * 100

with pd.option_context('display.max_rows', None):
 print(porcentagem_nulos)

employee_id              0.000
job_title                0.000
department               0.000
location                 0.000
salary                   0.000
hire_date                0.000
termination_date        79.948
is_active                0.000
absence_days             0.000
sick_days                0.000
vacation_days_taken      0.000
bank_hours               0.000
overtime_hours           0.000
tardiness_count          0.000
gender                   0.000
marital_status           0.000
number_of_dependents     0.000
education_level          0.000
performance_rating       0.000
bonus_percentage         0.000
shift                    0.000
contract_type            0.000
cost_center              0.000
compliance_status        0.000
health_plan              0.000
email                    0.000
tenure_years             0.000
probation_completed      0.000
manager_id               0.000
last_promotion_date     51.664
last_training_date      29.780
dtype: float64


In [109]:
#Verificando as colunas do df em lista
df.columns.tolist()

['employee_id',
 'job_title',
 'department',
 'location',
 'salary',
 'hire_date',
 'termination_date',
 'is_active',
 'absence_days',
 'sick_days',
 'vacation_days_taken',
 'bank_hours',
 'overtime_hours',
 'tardiness_count',
 'gender',
 'marital_status',
 'number_of_dependents',
 'education_level',
 'performance_rating',
 'bonus_percentage',
 'shift',
 'contract_type',
 'cost_center',
 'compliance_status',
 'health_plan',
 'email',
 'tenure_years',
 'probation_completed',
 'manager_id',
 'last_promotion_date',
 'last_training_date']

##**Tratamento para gerar o Glossário**

- Converter colunas de data para o tipo datetime
- Verificar se existem valores inconsistentes
- Identificar valores ausentes com `df.isnull().sum()`
- Criar o dicionário do glossário

##Análise inicial

In [111]:
#Criar cópia do df original
df1 = df.copy()

###Verificar % de nulos

In [112]:
#Verificar % de nulos
nulos = df1.isnull().mean().round(4) * 100
print(nulos)

employee_id              0.00
job_title                0.00
department               0.00
location                 0.00
salary                   0.00
hire_date                0.00
termination_date        79.95
is_active                0.00
absence_days             0.00
sick_days                0.00
vacation_days_taken      0.00
bank_hours               0.00
overtime_hours           0.00
tardiness_count          0.00
gender                   0.00
marital_status           0.00
number_of_dependents     0.00
education_level          0.00
performance_rating       0.00
bonus_percentage         0.00
shift                    0.00
contract_type            0.00
cost_center              0.00
compliance_status        0.00
health_plan              0.00
email                    0.00
tenure_years             0.00
probation_completed      0.00
manager_id               0.00
last_promotion_date     51.66
last_training_date      29.78
dtype: float64


###Validar se os valores nulos fazem sentido de fato com base nas outras colunas

Analisar pela termination_date, onde valores nulos devem ocorrer apenas quando is_active == True.

In [113]:
#Registros com termination_date nulo mas is_active == False (problema!)
inconsistentes_termination = df1[
    df1["termination_date"].isna() & (df1["is_active"] == False)
]

print(f"Nº de registros inconsistentes: {len(inconsistentes_termination)}")
inconsistentes_termination.head()


Nº de registros inconsistentes: 0


,employee_id,job_title,department,location,salary,hire_date,termination_date,is_active,absence_days,sick_days,vacation_days_taken,bank_hours,overtime_hours,tardiness_count,gender,marital_status,number_of_dependents,education_level,performance_rating,bonus_percentage,shift,contract_type,cost_center,compliance_status,health_plan,email,tenure_years,probation_completed,manager_id,last_promotion_date,last_training_date


Uma vez que o resultado é 0, podemos dizer que os nulos em termination_date são todos de funcionários ativos — ou seja, faz sentido manter como nulo.

Validação: last_promotion_date e last_training_date nulos
Essas duas colunas podem estar nulas em vários contextos:

- Nunca foi promovido

- Nunca participou de treinamento

Contabilizar quantos são ativos e nunca foram promovidos (o que pode ser normal):

In [114]:
#Promoção ausente e está ativo
sem_promocao = df1[
    df1["last_promotion_date"].isna() & (df1["is_active"] == True)
]

print(f"Nº de colaboradores ativos sem promoções: {len(sem_promocao)}")

Nº de colaboradores ativos sem promoções: 20653


Contabilizar quantos são ativos e não tiveram treinamento:

In [115]:
#Treinamento ausente e está ativo
sem_treinamento = df1[
    df1["last_training_date"].isna() & (df1["is_active"] == True)
]

print(f"Nº de colaboradores ativos sem treinamento: {len(sem_treinamento)}")

Nº de colaboradores ativos sem treinamento: 11945


##Tratamento da Base

###Trocar tipos datatime

In [116]:
#Corrigir os tipos de data no df1
date_columns = ['hire_date', 'termination_date', 'last_promotion_date', 'last_training_date']
for col in date_columns:
    df1[col] = pd.to_datetime(df1[col], errors='coerce')

In [117]:
#Listar colunas de data
date_columns = ['hire_date', 'termination_date', 'last_promotion_date', 'last_training_date']

#Criar df com tipos de dados antes e depois
comparacao_tipos = pd.DataFrame({
    'coluna': date_columns,
    'tipo_original': [df[col].dtype for col in date_columns],
    'tipo_corrigido': [df1[col].dtype for col in date_columns]
})

print(comparacao_tipos)


                coluna tipo_original  tipo_corrigido
0            hire_date        object  datetime64[ns]
1     termination_date        object  datetime64[ns]
2  last_promotion_date        object  datetime64[ns]
3   last_training_date        object  datetime64[ns]


###Alterar nomenclatura das colunas, para nome amigável

In [136]:
#Verificar tipos atualizados
tipos = df1.dtypes

In [137]:
#Estrutura base do glossário será utilizada depois também
glossario = pd.DataFrame({
    "Coluna original": df1.columns,
    "Coluna corrigida": [
        "ID do Colaborador", "Cargo", "Departamento", "Localização", "Salário", "Data de Admissão",
        "Data de Demissão", "Está Ativo", "Dias de Falta", "Dias de Licença Médica", "Dias de Férias Usufruídos",
        "Banco de Horas", "Horas Extras", "Quantidade de Atrasos", "Gênero", "Estado Civil",
        "Número de Dependentes", "Nível de Escolaridade", "Avaliação de Desempenho", "Percentual de Bônus", "Turno",
        "Tipo de Contrato", "Centro de Custo", "Status de Conformidade", "Plano de Saúde", "Email",
        "Tempo de Empresa (anos)", "Período de Experiência Concluído", "ID do Gestor",
        "Data da Última Promoção", "Data do Último Treinamento"
    ],
    "Tipo de Dado": tipos.astype(str).values
})

In [138]:
#Criar dicionário de renomeação
rename_dict = dict(zip(glossario["Coluna original"], glossario["Coluna corrigida"]))

#Renomear o DataFrame
df_final = df1.rename(columns=rename_dict)

#Verificar resultado
df_final.head()

,ID do Colaborador,Cargo,Departamento,Localização,Salário,Data de Admissão,Data de Demissão,Está Ativo,Dias de Falta,Dias de Licença Médica,Dias de Férias Usufruídos,Banco de Horas,Horas Extras,Quantidade de Atrasos,Gênero,Estado Civil,Número de Dependentes,Nível de Escolaridade,Avaliação de Desempenho,Percentual de Bônus,Turno,Tipo de Contrato,Centro de Custo,Status de Conformidade,Plano de Saúde,Email,Tempo de Empresa (anos),Período de Experiência Concluído,ID do Gestor,Data da Última Promoção,Data do Último Treinamento
0,EMP_00001,Gerente de RH,Financeiro,Salvador,6993.43,2022-02-26,NaT,True,3,2,13,52,20,6,Other,Widowed,4,Bachelors,5,7.58,Morning,Permanent,CC013,Compliant,Standard,emp_00001@company.com,3.34,True,EMP_00495,NaT,2024-07-04
1,EMP_00002,Engenheiro de Dados,Marketing,Belo Horizonte,5723.47,2023-03-04,NaT,True,1,1,14,-80,86,6,Male,Married,1,Bachelors,5,8.72,Morning,Temporary,CC026,Compliant,Basic,emp_00002@company.com,2.33,True,EMP_00345,2024-07-21,2024-02-01
2,EMP_00003,Desenvolvedor Python,Operações,Salvador,7295.38,2010-09-01,NaT,True,3,5,25,149,23,1,Female,Divorced,1,Masters,4,7.93,Morning,Permanent,CC010,Compliant,Basic,emp_00003@company.com,14.84,True,EMP_00306,2014-10-18,2024-01-11
3,EMP_00004,Analista Financeiro,Operações,Porto Alegre,9046.06,2020-02-26,NaT,True,2,1,21,-92,22,5,Male,Married,0,Bachelors,3,5.27,Morning,Permanent,CC008,Compliant,Standard,emp_00004@company.com,5.35,True,EMP_00971,NaT,2025-01-05
4,EMP_00005,Engenheiro de Dados,Recursos Humanos,Curitiba,5531.69,2011-06-22,NaT,True,1,2,15,93,72,3,Male,Single,0,Bachelors,2,10.95,Afternoon,Permanent,CC013,Compliant,Premium,emp_00005@company.com,14.03,True,EMP_00024,NaT,2025-01-19


2. **Normalização e padronização:**
    - Uniformizar valores categóricos (ex: unificar maiúsculas/minúsculas em `gender`, `marital_status`, `department`).
    - Padronizar formatos de texto e strings (ex: nomes, e-mails).

In [139]:
colunas_categoricas = [
    "Cargo", "Departamento", "Localização", "Gênero", "Estado Civil",
    "Nível de Escolaridade", "Turno", "Tipo de Contrato",
    "Status de Conformidade", "Plano de Saúde"
]

df_final[colunas_categoricas] = df_final[colunas_categoricas].applymap(
    lambda x: x.strip() if isinstance(x, str) else x
)


In [140]:
#Garantir que a coluna email possui todos os caracteres minúsculos
df_final["Email"] = df_final["Email"].str.strip().str.lower()

Tradução dos valores para pt-br

In [141]:
colunas_para_traduzir = [
    "Cargo", "Departamento", "Localização", "Gênero", "Estado Civil",
    "Nível de Escolaridade", "Turno", "Tipo de Contrato",
    "Compliance", "Plano de Saúde", "Centro de Custo", "Status de Conformidade"
]

In [142]:
#Mapeamento dos valores possíveis dentro das colunas a serem traduzidas
genero_map = {
    "Male": "Masculino",
    "Female": "Feminino",
    "Other": "Outro"
}

estado_civil_map = {
    "Single": "Solteiro(a)",
    "Married": "Casado(a)",
    "Divorced": "Divorciado(a)",
    "Widowed": "Viúvo(a)"
}

nivel_escolaridade_map = {
    "High School": "Ensino Médio",
    "Bachelors": "Graduação",
    "Masters": "Mestrado",
    "PhD": "Doutorado"
}

turno_map = {
    "Morning": "Manhã",
    "Afternoon": "Tarde",
    "Night": "Noite"
}

tipo_contrato_map = {
    "Permanent": "Permanente",
    "Temporary": "Temporário",
    "Contractor": "Prestador de Serviços"
}

compliance_map = {
    "Compliant": "Em conformidade",
    "Non-compliant": "Não conforme"
}

plano_saude_map = {
    "Basic": "Básico",
    "Standard": "Padrão",
    "Premium": "Premium"
}


In [143]:
df_final["Gênero"] = df_final["Gênero"].map(genero_map)
df_final["Estado Civil"] = df_final["Estado Civil"].map(estado_civil_map)
df_final["Nível de Escolaridade"] = df_final["Nível de Escolaridade"].map(nivel_escolaridade_map)
df_final["Turno"] = df_final["Turno"].map(turno_map)
df_final["Tipo de Contrato"] = df_final["Tipo de Contrato"].map(tipo_contrato_map)
df_final["Status de Conformidade"] = df_final["Status de Conformidade"].map(compliance_map)
df_final["Plano de Saúde"] = df_final["Plano de Saúde"].map(plano_saude_map)

Conversão de valores booleanos para textos descritivos

In [144]:
boolean_cols = ['Está Ativo', 'Período de Experiência Concluído']

for col in boolean_cols:
    df_final[col] = df_final[col].map({True: 'Sim', False: 'Não'})

###Validação de valores inválidos

Salários negativos

In [145]:
salarios_invalidos = df_final[df_final['Salário'] < 0]
print(f"Salários negativos encontrados: {len(salarios_invalidos)}")

Salários negativos encontrados: 0


Contagens negativas

In [146]:
contagem_cols = ['Dias de Falta', 'Dias de Licença Médica', 'Dias de Férias Usufruídos', 'Banco de Horas', 'Horas Extras', 'Quantidade de Atrasos']

for col in contagem_cols:
    invalidos = df_final[df_final[col] < 0]
    print(f"Valores negativos em {col}: {len(invalidos)}")


Valores negativos em Dias de Falta: 0
Valores negativos em Dias de Licença Médica: 0
Valores negativos em Dias de Férias Usufruídos: 0
Valores negativos em Banco de Horas: 16522
Valores negativos em Horas Extras: 0
Valores negativos em Quantidade de Atrasos: 0


###Datas incoerentes

In [147]:
#Promoção após demissão
promo_apos_demissao = df_final[(df_final['Data da Última Promoção'].notnull()) & (df_final['Data de Demissão'].notnull()) & (df_final['Data da Última Promoção'] > df_final['Data de Demissão'])]
print(f"Promoção após demissão: {len(promo_apos_demissao)}")

#Demissão antes da admissão
demissao_antes_admissao = df_final[(df_final['Data de Demissão'].notnull()) & (df_final['Data de Demissão'] < df_final['Data de Admissão'])]
print(f"Demissão antes da admissão: {len(demissao_antes_admissao)}")

#Treinamento antes da admissão
treinamento_antes_admissao = df_final[(df_final['Data do Último Treinamento'].notnull()) & (df_final['Data do Último Treinamento'] < df_final['Data de Admissão'])]
print(f"Treinamento antes da admissão: {len(treinamento_antes_admissao)}")


Promoção após demissão: 2902
Demissão antes da admissão: 0
Treinamento antes da admissão: 1321


###O que fazer com esses dados incoerentes?

###Analisar o impacto

Verificar quantos registros são no total e qual porcentagem representam os casos incoerentes:
- Promoção após demissão: 2902
- Demissão antes da admissão: 0
- Treinamento antes da admissão: 1321

###Tratamento
Algumas opções:

- Corrigir registros:
Identificar a causa e aplicar regras para correção;

- Manter com sinalização:
Criar campos que demarquem estas linhas, permitindo que os outros dados sejam aproveitados. Identificando os dados corretos na hora de realizar a análise no PBI.

Deixando claro, que esta opção foi escolhida apenas por se tratar de um Case Técnico onde não possuímos maiores informações sobre a captura destes dados. E também não podemos atuar na correção e na inclusão de regras para correção diretamente na fonte por também se tratar de uma base fictícia.

###Criar coluna que identifica estas datas, para serem retiradas do dataset na análise do PBI

Promoção após demissão

In [149]:
df_final['Flag: Promoção após Demissão'] = np.where(
    (df_final['Data da Última Promoção'].notnull()) &
    (df_final['Data de Demissão'].notnull()) &
    (df_final['Data da Última Promoção'] > df_final['Data de Demissão']),
    'Sim', 'Não'
)

Demissão antes da Admissão

In [150]:
df_final['Flag: Demissão antes da Admissão'] = np.where(
    (df_final['Data de Demissão'].notnull()) &
    (df_final['Data de Demissão'] < df_final['Data de Admissão']),
    'Sim', 'Não'
)

 Treinamento antes da admissão

In [151]:
df_final['Flag: Treinamento antes da Admissão'] = np.where(
    (df_final['Data do Último Treinamento'].notnull()) &
    (df_final['Data do Último Treinamento'] < df_final['Data de Admissão']),
    'Sim', 'Não'
)

In [153]:
df_final.head()

,ID do Colaborador,Cargo,Departamento,Localização,Salário,Data de Admissão,Data de Demissão,Está Ativo,Dias de Falta,Dias de Licença Médica,Dias de Férias Usufruídos,Banco de Horas,Horas Extras,Quantidade de Atrasos,Gênero,Estado Civil,Número de Dependentes,Nível de Escolaridade,Avaliação de Desempenho,Percentual de Bônus,Turno,Tipo de Contrato,Centro de Custo,Status de Conformidade,Plano de Saúde,Email,Tempo de Empresa (anos),Período de Experiência Concluído,ID do Gestor,Data da Última Promoção,Data do Último Treinamento,Flag: Promoção após Demissão,Flag: Demissão antes da Admissão,Flag: Treinamento antes da Admissão
0,EMP_00001,Gerente de RH,Financeiro,Salvador,6993.43,2022-02-26,NaT,Sim,3,2,13,52,20,6,Outro,Viúvo(a),4,Graduação,5,7.58,Manhã,Permanente,CC013,Em conformidade,Padrão,emp_00001@company.com,3.34,Sim,EMP_00495,NaT,2024-07-04,Não,Não,Não
1,EMP_00002,Engenheiro de Dados,Marketing,Belo Horizonte,5723.47,2023-03-04,NaT,Sim,1,1,14,-80,86,6,Masculino,Casado(a),1,Graduação,5,8.72,Manhã,Temporário,CC026,Em conformidade,Básico,emp_00002@company.com,2.33,Sim,EMP_00345,2024-07-21,2024-02-01,Não,Não,Não
2,EMP_00003,Desenvolvedor Python,Operações,Salvador,7295.38,2010-09-01,NaT,Sim,3,5,25,149,23,1,Feminino,Divorciado(a),1,Mestrado,4,7.93,Manhã,Permanente,CC010,Em conformidade,Básico,emp_00003@company.com,14.84,Sim,EMP_00306,2014-10-18,2024-01-11,Não,Não,Não
3,EMP_00004,Analista Financeiro,Operações,Porto Alegre,9046.06,2020-02-26,NaT,Sim,2,1,21,-92,22,5,Masculino,Casado(a),0,Graduação,3,5.27,Manhã,Permanente,CC008,Em conformidade,Padrão,emp_00004@company.com,5.35,Sim,EMP_00971,NaT,2025-01-05,Não,Não,Não
4,EMP_00005,Engenheiro de Dados,Recursos Humanos,Curitiba,5531.69,2011-06-22,NaT,Sim,1,2,15,93,72,3,Masculino,Solteiro(a),0,Graduação,2,10.95,Tarde,Permanente,CC013,Em conformidade,Premium,emp_00005@company.com,14.03,Sim,EMP_00024,NaT,2025-01-19,Não,Não,Não


###Criar glossário

In [159]:
# Estrutura base do glossário
glossario = pd.DataFrame({
    "Coluna Original": df1.columns,
    "Coluna corrigida": [
        "ID do Colaborador", "Cargo", "Departamento", "Localização", "Salário", "Data de Admissão",
        "Data de Demissão", "Está Ativo", "Dias de Falta", "Dias de Licença Médica", "Dias de Férias Usufruídos",
        "Banco de Horas", "Horas Extras", "Quantidade de Atrasos", "Gênero", "Estado Civil",
        "Número de Dependentes", "Nível de Escolaridade", "Avaliação de Desempenho", "Percentual de Bônus", "Turno",
        "Tipo de Contrato", "Centro de Custo", "Status de Conformidade", "Plano de Saúde", "Email",
        "Tempo de Empresa (anos)", "Período de Experiência Concluído", "ID do Gestor",
        "Data da Última Promoção", "Data do Último Treinamento"
    ],
    "Tipo de Dado": tipos.astype(str).values
})

In [161]:
# Criar base do glossário considerando todas as colunas do df_final
glossario = pd.DataFrame({
    "Coluna corrigida": list(df_final.columns)
})

# Adicionar Coluna Original com base na ordem do df_corrigido (pode ter menos colunas)
glossario["Coluna Original"] = pd.Series(df1.columns)

# Adicionar os tipos de dados
glossario["Tipo de Dado"] = df_final.dtypes.astype(str).values

# Reorganizar colunas
glossario = glossario[["Coluna Original", "Coluna corrigida", "Tipo de Dado"]]

# Exibir resultado
display(glossario)


,Coluna Original,Coluna corrigida,Tipo de Dado
0,employee_id,ID do Colaborador,object
1,job_title,Cargo,object
2,department,Departamento,object
3,location,Localização,object
4,salary,Salário,float64
5,hire_date,Data de Admissão,datetime64[ns]
6,termination_date,Data de Demissão,datetime64[ns]
7,is_active,Está Ativo,object
8,absence_days,Dias de Falta,int64
9,sick_days,Dias de Licença Médica,int64


In [162]:
#Filtrar apenas colunas categóricas (tipo object ou category)
colunas_categoricas = df_final.select_dtypes(include=['object', 'category']).columns

In [163]:
#Dicionário com valores únicos de colunas categóricas
valores_categoricos = {
    coluna: df_final[coluna].dropna().unique().tolist()
    for coluna in colunas_categoricas
}

In [164]:
#Mostrar exemplo de valores únicos por coluna
for coluna, valores in valores_categoricos.items():
    print(f"{coluna}: {valores[:20]}")

ID do Colaborador: ['EMP_00001', 'EMP_00002', 'EMP_00003', 'EMP_00004', 'EMP_00005', 'EMP_00006', 'EMP_00007', 'EMP_00008', 'EMP_00009', 'EMP_00010', 'EMP_00011', 'EMP_00012', 'EMP_00013', 'EMP_00014', 'EMP_00015', 'EMP_00016', 'EMP_00017', 'EMP_00018', 'EMP_00019', 'EMP_00020']
Cargo: ['Gerente de RH', 'Engenheiro de Dados', 'Desenvolvedor Python', 'Analista Financeiro', 'Coordenador de TI', 'Auxiliar Administrativo', 'Consultor de BI', 'Analista de Dados', 'Diretor de Operações', 'Estagiário']
Departamento: ['Financeiro', 'Marketing', 'Operações', 'Recursos Humanos', 'Jurídico', 'TI', 'Vendas']
Localização: ['Salvador', 'Belo Horizonte', 'Porto Alegre', 'Curitiba', 'São Paulo', 'Fortaleza', 'Rio de Janeiro']
Está Ativo: ['Sim', 'Não']
Gênero: ['Outro', 'Masculino', 'Feminino']
Estado Civil: ['Viúvo(a)', 'Casado(a)', 'Divorciado(a)', 'Solteiro(a)']
Nível de Escolaridade: ['Graduação', 'Mestrado', 'Ensino Médio', 'Doutorado']
Turno: ['Manhã', 'Tarde', 'Noite']
Tipo de Contrato: ['Perma

In [172]:
# Atualizar observações do glossário
glossario["Observações"] = [
    "Identificação do colaborador = Ex: EMP_00001, EMP_OOOO2",
    "Gerente de RH, Engenheiro de Dados, Desenvolvedor Python, Analista Financeiro, Coordenador de TI, Auxiliar Administrativo, Consultor de BI, Analista de Dados, Diretor de Operações, Estagiário",
    "Financeiro, Marketing, Operações, Recursos Humanos, Jurídico, TI, Vendas",
    "Salvador, Belo Horizonte, Porto Alegre, Curitiba, São Paulo, Fortaleza, Rio de Janeiro",
    "Valor contínuo em reais",
    "Convertido para datetime",
    "",
    "'Sim' - indica que está ativo, 'Não' - indica que foi desligado",
    "Faltas injustificadas e justificadas",
    "",
    "",
    "Pode conter valores negativos (uso ou dívida)",
    "Horas extras acumuladas",
    "Contagem de atrasos registrados",
    "Categoria: Feminino, Masculino e Outro",
    "Categoria: Viúvo(a), Casado(a), Divorciado(a), Solteiro(a)",
    "Número inteiro",
    "Categoria: Graduação, Mestrado, Ensino Médio, Doutorado",
    "Nota de 1 a 5",
    "Percentual de bônus - contínuo",
    "Turno de trabalho: Manhã, Tarde, Noite",
    "Tipo contratual: Permanente, Temporário",
    "Código de centro de custo",
    "Status de conformidade",
    "Premium, Básico, Padrão",
    "Formato padrão de e-mail",
    "Anos de empresa (float)",
    "Sim, Não",
    "ID do gestor (Chave Gestor)",
    "Nulos indicam sem promoção registrada",
    "Nulos indicam ausência de treinamentos recentes",
    "Indica registros em que a data da última promoção é posterior à data de demissão, o que é logicamente inconsistente.",
    "Identifica casos onde a data de demissão ocorre antes da admissão, o que aponta para erro nos dados históricos",
    "Aponta registros em que a data do último treinamento ocorreu antes da admissão, o que pode sugerir erro de cadastro ou transferência interna sem ajuste de datas"
]


In [173]:
#Exibir Glossário criado
display(glossario)

,Coluna Original,Coluna corrigida,Tipo de Dado,Observações
0,employee_id,ID do Colaborador,object,"Identificação do colaborador = Ex: EMP_00001, ..."
1,job_title,Cargo,object,"Gerente de RH, Engenheiro de Dados, Desenvolve..."
2,department,Departamento,object,"Financeiro, Marketing, Operações, Recursos Hum..."
3,location,Localização,object,"Salvador, Belo Horizonte, Porto Alegre, Curiti..."
4,salary,Salário,float64,Valor contínuo em reais
5,hire_date,Data de Admissão,datetime64[ns],Convertido para datetime
6,termination_date,Data de Demissão,datetime64[ns],
7,is_active,Está Ativo,object,"'Sim' - indica que está ativo, 'Não' - indica ..."
8,absence_days,Dias de Falta,int64,Faltas injustificadas e justificadas
9,sick_days,Dias de Licença Médica,int64,


###Salvar o glossário

In [174]:
#Salvar o glossário em CSV localmente no ambiente do Colab
glossario.to_csv("glossario.csv", index=False)

#Fazer o download do arquivo para o seu computador
files.download("glossario.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Exportar com separador decimal vírgula (excel/PT-BR) e ponto e vírgula como separador de coluna
df_final.to_csv("base_corrigida.csv", sep=';', decimal=',', index=False, encoding='utf-8')

#Baixar o arquivo
files.download("base_corrigida.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>